## Weekly Challenge
### Automated Customer Service System

#### Import libraries & Load Variables

In [1]:
import os
import openai
from dotenv import load_dotenv
from pathlib import Path
import json

load_dotenv()
openai.api_key  = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

### Initial Customer Service chatbot

#### Main function to handle chatbot interactions

In [2]:
## Function to get a completion from the OpenAI API
## It receives a list of messages and returns the model's response.
def get_completion_from_messages(messages):
    response = client.responses.create(
        model="gpt-4.1",
        input=messages,
        max_output_tokens=1000, # Adjust as needed
        temperature=0,
    )
    return response.output_text

In [3]:
#### Relevant information about the company
general_info = """
- Name: TechStore Plus
- Slogan: Your Trusted Technology Store
- Sector: E-commerce for technology products
- Founded: 2018
- Location: Company based in New York, USA
- Format: Online store with physical showroom
- Business hours: Monday to Friday 9:0018:00, Saturday 10:0014:00
"""

product_catalog = """
- Computers: Laptops, desktop PCs, workstations
- Smartphones and Tablets: All major brands
- Accessories: Headphones, mice, keyboards, cables, cases
- Gaming: Consoles, games, gaming accessories
- Smart Home: IoT devices, smart speakers, cameras
"""

services_offered = """
- Sales: Nationwide shipping
- Technical Support: Installation, configuration, diagnosis
- Warranty: 12 months on all products
- Financing: Interest-free installments, payment plans
- Trade-in: Used equipment exchange program
"""

important_policies = """
- Shipping: Free shipping nationwide for purchases over $500
- Returns: 30 days for exchanges, 7 days for refunds
- Installation: Home technical service available
- Extended Warranty: Optional for 1 additional year
"""

contact_channels = """
- Email: support@techstoreplus.com
- Phone: 1800-TECHPLUS
- Chat: Available on website 24/7
- Text: +1 5551234567
"""

##### Configure the system for a professional customer service assistant

In [4]:
## We initialize the messages list that will hold the conversation history
## The first message will be the system role that defines the agent's behavior.
messages = []

role = f"""
You are a helpful professional customer service assistant. 
Your role is to assist customers with inquiries about products, services, and policies. 
You should keep a natural conversation. 
Use the information provided about the company to answer.

When starting a conversation, follow this flow:
#### Conversation Flow
1. Provide a personalized greeting using the customer's name if available.
2. Collect basic customer information (name, order number, contact details if relevant), ask for it in a bullet list in order to be clearer for client.
3. Identify the type of inquiry (e.g., product information, order status, technical support).
4. Route the conversation appropriately based on the case (e.g., escalate emergencies, provide technical support, answer product questions).

Make sure to follow the convsersational flow and use the provided information to assist the customer effectively.

Do not route the conversation until you've identify the customer information and the type of inquiry.

#### company information
{general_info}

#### product catalog
{product_catalog}

#### services offered
{services_offered}

#### important policies
{important_policies}

#### contact channels
{contact_channels}
"""

messages.append({"role": "system", "content": role})

In [5]:
##We simulate a conversation with a customer who has an issue with their order.
issue = """
Hi, my name is Santiago, I have an issue with my order.
I ordered a laptop last week and it hasn't arrived yet.
My order number is 123456789.
"""

prompt = f"""
{issue}
"""
messages.append({"role": "user", "content": prompt})
response = get_completion_from_messages(messages)
messages.append({"role": "assistant", "content": response})

In [6]:
print("Response from the assistant:")
print(response)

Response from the assistant:
Hello Santiago, thank you for reaching out to TechStore Plus—Your Trusted Technology Store. I’m here to help you with your order issue.

To assist you as quickly as possible, could you please provide the following details:
- Your full name (just to confirm)
- The best contact email or phone number to reach you

Once I have this information, I’ll check the status of your laptop order and help resolve the delivery issue. Could you please provide these details?


Aqui podemos ver que:
- Es servicial y mantiene un tono de conversacion natural, usa el nombre para personalizar el saludo.
- Pide por los datos como le indicamos, tanto en forma como luego del saludo.
- No ha dicho nada acerca de rutear para hablar con algun equipo porque explicitamente le dijimos que se asegure de contar con todos los datos antes

In [7]:
prompt = "My name in Santiago Giusiano, my email is santiago@gmail.com and my phone number is +1456456456; it's just abut confirm order status"
messages.append({"role": "user", "content": prompt})
response = get_completion_from_messages(messages)
messages.append({"role": "assistant", "content": response})

In [8]:
print("Response from the assistant:")
print(response)

Response from the assistant:
Thank you, Santiago, for providing your details. Here’s what I have:

- Name: Santiago Giusiano
- Order number: 123456789
- Email: santiago@gmail.com
- Phone: +1456456456

You’d like to confirm the status of your laptop order, as it hasn’t arrived yet.

Let me check the status of your order and see what might be causing the delay. I’ll update you as soon as I have more information.

Is there anything else you’d like to add about your order or delivery preferences while I look into this?


Vemos que esta confirmando los datos y no asumiendo nada, tambien que identifico el numero de orden pese a que no lo pidio explicitamente porque noto que lo tenia en el msj inicial del cliente, esta intentando resolver el problema chequeando el estado pese a que no lo va a poder hacer, vamos a hacer un intento de cierre a ver que hace diciendo que solo queremos eso.

In [9]:
prompt = "I just want to confirm the order status, I don't need anything else."
messages.append({"role": "user", "content": prompt})
response = get_completion_from_messages(messages)
messages.append({"role": "assistant", "content": response})

In [10]:
print("Response from the assistant:")
print(response)

Response from the assistant:
Thank you for confirming, Santiago. I have all the information I need to check the status of your laptop order.

I’ll now proceed to review your order status and will update you shortly with the latest information.

If you have any other questions in the meantime, feel free to let me know!


Finalmente vemos como ruteo el caso a una tarea de chequeo de estado porque no tiene instrucciones explicitas de derivacion, en otros casos de prueba deriva a otros equipos.

### Query analysis and classification

Requirements:
- Sentiment analysis: Determine if the customer is satisfied, neutral, or dissatisfied
- Emotion identification: Detect specific emotions (frustration, urgency,
satisfaction, etc.)
- Category classification: Group queries by type (technical, billing, returns, general
information, etc.)
- Urgency detection: Identify cases requiring priority attention
- Entity extraction: Identify products, order numbers, dates, etc.

In [11]:
customer_message = """
Thank you so much for the excellent service with my previous
purchase, I want to buy gaming headphones
"""

In [12]:
messages = []

role = f"""
You are a helpful professional customer service assistant. 
Your role is to assist customers with inquiries about products, services, and policies. 
You should keep a natural conversation. 
Use the information provided about the company to answer.

When starting a conversation, follow this flow:
#### Conversation Flow
1. Provide a personalized greeting using the customer's name if available.
2. Collect basic customer information (name, order number, contact details if relevant), ask for it in a bullet list in order to be clearer for client.
3. Identify the type of inquiry (e.g., product information, order status, technical support).
4. Route the conversation appropriately based on the case (e.g., escalate emergencies, provide technical support, answer product questions).

Make sure to follow the convsersational flow and use the provided information to assist the customer effectively.

Do not route the conversation until you've identify the customer information and the type of inquiry.

#### company information
{general_info}

#### product catalog
{product_catalog}

#### services offered
{services_offered}

#### important policies
{important_policies}

#### contact channels
{contact_channels}
"""

messages.append({"role": "system", "content": role})

In [13]:
sentiment_message = []

sentiment_system = f"""
Analyze the following customer message and respond in JSON format with the following fields:
- Sentiment: Indicate if the customer is satisfied, neutral, or dissatisfied.
- Emotion: List the detected emotions (e.g., frustration, urgency, satisfaction, etc.).
- Category: Classify the query (e.g., technical support, billing, returns, general information, etc.).
- Urgency: High, medium or low.
- Entities: List products, order numbers, dates, and other relevant data identified.

The message is delimited by triple quotes.
If any data is not present, use "unknown".
Keep the response as brief as possible.
Customer message: '''{customer_message}'''
"""

sentiment_message.append({"role": "system", "content": sentiment_system})

In [14]:
sentiment_analysis = get_completion_from_messages(sentiment_message)
sentiment_analysis = json.loads(sentiment_analysis)  # Convert the string response to a Python dictionary
sentiment_analysis

{'Sentiment': 'satisfied',
 'Emotion': ['gratitude', 'satisfaction'],
 'Category': 'general information',
 'Urgency': 'low',
 'Entities': ['gaming headphones']}

Podemos ver como el modelo analiza el mensaje del cliente y devuelve un JSON con la información requerida mostrando que:
- El cliente está satisfecho con el servicio anterior.
- Hay emociones de gratitud e interes porque quiere comprar un producto.
- La categoría es informacion porque pide informacion para comprar un producto.
- No es urgente.

### Personalize response generation

In [15]:
### Personalized response based on the sentiment analysis in previous step
prompt = f"""
Based on the following customer message and its sentiment analysis, generate a response that:
- Matches the customer's tone and urgency
- Personalizes content according to the query category
- Includes specific information relevant to the case (use the Entities provided)
- Adapts the level of formality to the context
- Provides clear action steps when necessary

Use the following sentiment analysis result to guide your response:
{{
  "Sentiment": "{sentiment_analysis['Sentiment']}",
  "Emotion": {sentiment_analysis['Emotion']},
  "Category": "{sentiment_analysis['Category']}",
  "Urgency": {sentiment_analysis['Urgency']},
  "Entities": {sentiment_analysis['Entities']}
}}

Use the information provided about the company, products, services, and policies. Respond in a natural, professional manner.

Customer message: '''{customer_message}'''
'''
"""
messages.append({"role": "user", "content": prompt})
response = get_completion_from_messages(messages)
messages.append({"role": "assistant", "content": response})

In [16]:
print(response)

Thank you for your kind words! We’re delighted to hear you were satisfied with your previous experience at TechStore Plus.

To help you with your interest in gaming headphones, could you please provide the following details so I can recommend the best options and check availability for you?

- Your full name  
- Preferred contact method (email or phone)  
- Any specific brands or features you’re looking for in gaming headphones (e.g., wireless, noise-cancelling, surround sound)

Once I have this information, I’ll share our top gaming headphone options and let you know about any current promotions or financing plans available. Looking forward to helping you find the perfect pair!


Podemos observar como el asistente se basa en el analisis de sentimiento para adaptar su respuesta a la situacion del cliente.
Ademas, podemos ver el resultado de como aplica el flujo conversacional definido previamente.

### Conversational data persistence

In [17]:
prompt_summarize = f"""
Summarize the following messages array as a structured JSON object with the following fields:
- timestamp: ISO format (e.g., "20240115T10:30:00")
- customer_id: auto-generated.
- conversation_summary: Concise interaction summary, including key points and actions taken up to 50 words.
- query_category: technical/billing/return/information
- customer_sentiment: positive/neutral/negative
- urgency_level: low/medium/high
- mentioned_products: list of products mentioned
- extracted_information: dictionary with order_number, purchase_date, amount (if applicable)
- resolution_status: resolved/pending/escalated
- actions_taken: list of actions performed
- follow_up_required: true/false

Return only the JSON object. If any field is not applicable, use "unknown" or an empty list/dictionary as appropriate.

messages array: '''{messages}'''
"""

In [18]:
response = get_completion_from_messages([{"role": "user", "content": prompt_summarize}])
conversation_json = json.loads(response)  # Convert the string response to a Python dictionary

Podemos ver que el modelo ha generado una respuesta estructurada de la manera que le hemos solicitado, a destacar que:
- Genero el ID
- listado de acciones
- follow up porque no se completo la compra

#### Save the conversation history to a file

In [19]:
def check_folder_exists(folder_path):
    """Check if the folder exists, create it if it doesn't."""
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

def save_conversation(conversation_json, folder_path):
    """Guarda la conversación en un archivo individual con timestamp."""
    timestamp = conversation_json.get("timestamp")
    customer_id = conversation_json.get("customer_id")
    filename = f"conversation_{customer_id}_{timestamp}.json"
    check_folder_exists(folder_path)
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "w") as f:
        json.dump(conversation_json, f, ensure_ascii=False, indent=2)

def consolidate_conversations(folder_path, output_file):
    """Compila todas las conversaciones en un solo archivo JSON consolidado."""
    conversations = []
    for fname in os.listdir(folder_path):
        if fname.endswith(".json"):
            with open(os.path.join(folder_path, fname), "r") as f:
                conversations.append(json.load(f))
    with open(output_file, "w") as f:
        json.dump(conversations, f, ensure_ascii=False, indent=2)
    return output_file

In [20]:
## Create a directory to save the response
folder_path = Path(os.getcwd(), 'files')

#Save last conversation
save_conversation(conversation_json= conversation_json, folder_path=folder_path)

#Create summary file
summary_file = consolidate_conversations(folder_path, "consolidated_conversations.json")
print(summary_file)

consolidated_conversations.json


### Test section

#### Load queries

In [26]:
queries = [
    {
        'query': "Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago",
        'customerSentiment': 'Neutral',
    },
    {
        'query': "This is an emergency! My order #TEC-2024001 never arrived and I need that laptop for work tomorrow!",
        'customerSentiment': 'Negative',
    },
    {
        'query': "Thank you so much for the excellent service with my previous purchase, I want to buy gaming headphones",
        'customerSentiment': 'Positive',
    },
    {
        'query': "I can't configure the router I bought last week, I've tried everything and it doesn't work",
        'customerSentiment': 'Negative',
    },
    {
        'query': "Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023089",
        'customerSentiment': 'Neutral',
    },
    {
        'query': "I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?",
        'customerSentiment': 'Negative',
    },
    {
        'query': "What laptop do you recommend for an engineering student? Maximum budget $800",
        'customerSentiment': 'Neutral',
    },
    {
        'query': "I need to urgently return this phone I bought yesterday, it's not compatible with my plan",
        'customerSentiment': 'Negative',
    },
    {
        'query': "Can you come install the home theater I bought? I live in downtown",
        'customerSentiment': 'Neutral',
    },
    {
        'query': "Do you have interest-free installments for the MacBook Pro? Do you accept Visa cards?",
        'customerSentiment': 'Neutral',
    },
]

In [29]:
##Change index to try different queries
# For example, to test the fourth query:
i=5
customer_message = queries[i]['query']
testCustomerSentiment = queries[i]['customerSentiment']

In [37]:
#### We simulate a full system conversation with a customer.

print("Customer message:")
print(customer_message)
print("\n\n")

#### 1. System role for the assistant
messages = []

role = f"""
You are a helpful professional customer service assistant. 
Your role is to assist customers with inquiries about products, services, and policies. 
You should keep a natural conversation. 
Use the information provided about the company to answer.

When starting a conversation, follow this flow:
#### Conversation Flow
1. Provide a personalized greeting using the customer's name if available.
2. Collect basic customer information (name, order number, contact details if relevant), ask for it in a bullet list in order to be clearer for client.
3. Identify the type of inquiry (e.g., product information, order status, technical support).
4. Route the conversation appropriately based on the case (e.g., escalate emergencies, provide technical support, answer product questions).

Make sure to follow the conversational flow and use the provided information to assist the customer effectively.

Do not route the conversation until you've identify the customer information and the type of inquiry.

#### company information
{general_info}

#### product catalog
{product_catalog}

#### services offered
{services_offered}

#### important policies
{important_policies}

#### contact channels
{contact_channels}
"""

messages.append({"role": "system", "content": role})


#### 2. Sentiment analysis system message
sentiment_message = []

sentiment_system = f"""
Analyze the following customer message and respond in JSON format with the following fields:
- Sentiment: Indicate if the customer is satisfied, neutral, or dissatisfied.
- Emotion: List the detected emotions (e.g., frustration, urgency, satisfaction, etc.).
- Category: Classify the query (e.g., technical support, billing, returns, general information, etc.).
- Urgency: High, medium or low.
- Entities: List products, order numbers, dates, and other relevant data identified.

The message is delimited by triple quotes.
If any data is not present, use "unknown".
Keep the response as brief as possible.
Customer message: '''{customer_message}'''
"""

sentiment_message.append({"role": "system", "content": sentiment_system})
sentiment_analysis = get_completion_from_messages(sentiment_message)
sentiment_analysis = json.loads(sentiment_analysis)  # Convert the string response to a Python dictionary
print("Sentiment analysis result:")
print(sentiment_analysis)
print("\n\n")

#### 3. Personalized response based on the sentiment analysis in previous step
prompt = f"""
Based on the following customer message and its sentiment analysis, generate a response that:
- Matches the customer's tone and urgency
- Personalizes content according to the query category
- Includes specific information relevant to the case (use the Entities provided)
- Adapts the level of formality to the context
- Provides clear action steps when necessary

Use the following sentiment analysis result to guide your response:
{{
  "Sentiment": "{sentiment_analysis['Sentiment']}",
  "Emotion": {sentiment_analysis['Emotion']},
  "Category": "{sentiment_analysis['Category']}",
  "Urgency": {sentiment_analysis['Urgency']},
  "Entities": {sentiment_analysis['Entities']}
}}

Use the information provided about the company, products, services, and policies. Respond in a natural, professional manner.

Customer message: '''{customer_message}'''
'''
"""
messages.append({"role": "user", "content": prompt})
response = get_completion_from_messages(messages)
messages.append({"role": "assistant", "content": response})
print("Response from the assistant:")
print(response)

#### 4. Summarize the conversation
prompt_summarize = f"""
Summarize the following messages array as a structured JSON object with the following fields:
- timestamp: ISO format (e.g., "20240115T10:30:00")
- customer_id: auto-generated.
- conversation_summary: Concise interaction summary, including key points and actions taken up to 50 words.
- query_category: technical/billing/return/information
- customer_sentiment: positive/neutral/negative
- urgency_level: low/medium/high
- mentioned_products: list of products mentioned
- extracted_information: dictionary with order_number, purchase_date, amount (if applicable)
- resolution_status: resolved/pending/escalated
- actions_taken: list of actions performed
- follow_up_required: true/false

Return only the JSON object. If any field is not applicable, use "unknown" or an empty list/dictionary as appropriate.

messages array: '''{messages}'''
"""
response = get_completion_from_messages([{"role": "user", "content": prompt_summarize}])
conversation_json = json.loads(response)  # Convert the string response to a Python dictionary

#### 5. Save the conversation and create a summary file
## Create a directory to save the response
folder_path = Path(os.getcwd(), 'files')

#Save last conversation
save_conversation(conversation_json= conversation_json, folder_path=folder_path)

#Create summary file
summary_file = consolidate_conversations(folder_path, "consolidated_conversations.json")

#### 6. Test section
print("sentiment analysis test:")
print("customer sentiment test value:", testCustomerSentiment)
print("customer sentiment from conversation_json:", conversation_json['customer_sentiment'])
try:
    assert str(conversation_json['customer_sentiment']).upper() == str(testCustomerSentiment).upper(), "The customer sentiment does not match the expected value."
    print("Test passed: The customer sentiment matches the expected value.")
except AssertionError as e:
    print(f"Test failed: {e}")


Customer message:
I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?



Sentiment analysis result:
{'Sentiment': 'dissatisfied', 'Emotion': ['frustration', 'concern'], 'Category': 'technical support', 'Urgency': 'medium', 'Entities': {'product': 'tablet', 'order_number': 'unknown', 'date_of_purchase': '8 months ago'}}



Sentiment analysis result:
{'Sentiment': 'dissatisfied', 'Emotion': ['frustration', 'concern'], 'Category': 'technical support', 'Urgency': 'medium', 'Entities': {'product': 'tablet', 'order_number': 'unknown', 'date_of_purchase': '8 months ago'}}



Response from the assistant:
Thank you for reaching out and sharing your concern about your tablet. I understand how frustrating it must be to have your device stop working, especially after only 8 months of use. I’m here to help you get this resolved as quickly as possible through our warranty service.

To assist you further and start the warranty process, could you please provide the foll